Based on https://bambinos.github.io/bambi/notebooks/quantile_regression.html

In [ ]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
from data_utils import download_single_csv
from pathlib import Path

In [ ]:
az.style.use("arviz-darkgrid")
SEED = 12947

In [ ]:
download_single_csv(Path("./data"), "bmi.csv")

In [ ]:
x = np.linspace(-6, 6, 2000)
quantiles = np.array([0.2, 0.5, 0.8])
for q, m in zip(quantiles, [0, 0, -1]):
    κ = (q / (1 - q)) ** 0.5
    plt.plot(x, stats.laplace_asymmetric(κ, m, 1).pdf(x), label=f"q={q:}, μ={m}, σ=1")
plt.yticks([])
plt.legend();

In [ ]:
data = pd.read_csv("data/bmi.csv")
data.head()

In [ ]:
plt.plot(data.age, data.bmi, "k.")
plt.xlabel("Age")
plt.ylabel("BMI");

In [ ]:
data

In [ ]:
quantiles = np.array([0.1, 0.5, 0.9])
kappas = (quantiles / (1 - quantiles)) ** 0.5

knots = np.quantile(data.age, np.linspace(0, 1, 10))[1:-1]

idatas = []
for κ in kappas:
    model = bmb.Model(
        "bmi ~ bs(age, knots=knots)",
        data=data,
        family="asymmetriclaplace",
        priors={"kappa": κ},
    )
    idata = model.fit()
    model.predict(idata)
    idatas.append(idata)

Above cell broken: 
```
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
Cell In[19], line 11
      8 print(f"{κ=}")
      9 model = bmb.Model("bmi ~ bs(age, knots=knots)",
     10                   data=data.sample(n=200,replace=False), family="asymmetriclaplace", priors={"kappa": κ})
---> 11 idata = model.fit()
     12 model.predict(idata)
     13 idatas.append(idata)

File ~/PetProjects/testing-bambi/.venv/lib/python3.13/site-packages/bambi/models.py:330, in Model.fit(self, draws, tune, discard_tuned_samples, omit_offsets, include_mean, include_response_params, inference_method, init, n_init, chains, cores, random_seed, **kwargs)
    327         inference_method = method
    329 if not self.built:
--> 330     self.build()
    332 # Tell user which event is being modeled
    333 if isinstance(self.family, univariate.Bernoulli):

File ~/PetProjects/testing-bambi/.venv/lib/python3.13/site-packages/bambi/models.py:373, in Model.build(self)
    364 """Set up the model for sampling/fitting.
    365 
    366 Creates an instance of the underlying PyMC model and adds all the necessary terms to it.
   (...)
    370 None
    371 """
    372 self.backend = PyMCModel()
--> 373 self.backend.build(self)
    374 self.built = True

File ~/PetProjects/testing-bambi/.venv/lib/python3.13/site-packages/bambi/backend/pymc.py:89, in PyMCModel.build(self, spec)
     87 # Add response
     88 self.response_component = ResponseComponent(spec.response_component)
---> 89 self.response_component.build(self, spec)
     91 # Add potentials
     92 self.build_potentials(spec)

File ~/PetProjects/testing-bambi/.venv/lib/python3.13/site-packages/bambi/backend/model_components.py:168, in ResponseComponent.build(self, pymc_backend, bmb_model)
    165 def build(self, pymc_backend, bmb_model):
    166     # Create and build the response term
    167     response_term = ResponseTerm(self.component.term, bmb_model.family)
--> 168     response_term.build(pymc_backend, bmb_model)

File ~/PetProjects/testing-bambi/.venv/lib/python3.13/site-packages/bambi/backend/terms.py:275, in ResponseTerm.build(self, pymc_backend, bmb_model)
    272     kwargs[name] = pm.Deterministic(aliased_name, linkinv(output), dims=dims)
    274 # Build the response distribution
--> 275 dist = self.build_response_distribution(kwargs, pymc_backend)
    277 return dist

File ~/PetProjects/testing-bambi/.venv/lib/python3.13/site-packages/bambi/backend/terms.py:383, in ResponseTerm.build_response_distribution(self, kwargs, pymc_backend)
    380     dist_rv = weighted_dist(self.name, weights, **kwargs, observed=observed, dims=dims)
    381 # All of the other response kinds are "not special" and thus are handled the same way
    382 else:
--> 383     dist_rv = distribution(self.name, **kwargs)
    385 return dist_rv

File ~/PetProjects/testing-bambi/.venv/lib/python3.13/site-packages/pymc/distributions/distribution.py:511, in Distribution.__new__(cls, name, rng, dims, initval, observed, total_size, transform, default_transform, *args, **kwargs)
    508     elif observed is not None:
    509         kwargs["shape"] = tuple(observed.shape)
--> 511 rv_out = cls.dist(*args, **kwargs)
    513 rv_out = model.register_rv(
    514     rv_out,
    515     name,
   (...)
    521     initval=initval,
    522 )
    524 # add in pretty-printing support

File ~/PetProjects/testing-bambi/.venv/lib/python3.13/site-packages/pymc/distributions/continuous.py:1600, in AsymmetricLaplace.dist(cls, kappa, mu, b, q, *args, **kwargs)
   1597 @classmethod
   1598 def dist(cls, kappa=None, mu=None, b=None, q=None, *args, **kwargs):
   1599     kappa = cls.get_kappa(kappa, q)
-> 1600     return super().dist([b, kappa, mu], *args, **kwargs)

File ~/PetProjects/testing-bambi/.venv/lib/python3.13/site-packages/pymc/distributions/distribution.py:579, in Distribution.dist(cls, dist_params, shape, **kwargs)
    575 if ndim_supp is None:
    576     # Initialize Ops and check the ndim_supp that is now required to exist
    577     ndim_supp = cls.rv_op(*dist_params, **kwargs).owner.op.ndim_supp
--> 579 create_size = find_size(shape=shape, size=size, ndim_supp=ndim_supp)
    580 rv_out = cls.rv_op(*dist_params, size=create_size, **kwargs)
    582 _add_future_warning_tag(rv_out)

File ~/PetProjects/testing-bambi/.venv/lib/python3.13/site-packages/pymc/distributions/shape_utils.py:200, in find_size(shape, size, ndim_supp)
    198 if shape is not None:
    199     ndim_expected = len(tuple(shape))
--> 200     ndim_batch = ndim_expected - ndim_supp
    201     return tuple(shape)[:ndim_batch]
    203 return None

TypeError: unsupported operand type(s) for -: 'int' and 'method'
```